<a href="https://colab.research.google.com/github/shaunjohann/sjLlamaIndex/blob/main/Advanced_Tools_Usage_with_GSuite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google combined tools example

This notebook features a more advanced usage of Agent Tools, using the Google Calendar, Mail and Search integrations as well as the Load and Search Meta Tool to fufill a more complicated set of tasks for the user.

## Setup the Tools
First we will import OpenAI and setup the Agent:

In [ ]:
!pip install llama-index==0.7.8 llama-hub==0.0.11

In [ ]:
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

In [ ]:
from llama_index.agent import OpenAIAgent

In [ ]:
import openai

openai.api_key = 'sk-api-key'

Now we can import the Google Tools we are going to use. See the README for the respective tools to get started with authentication.

Gmail: https://llama-hub-ui.vercel.app/l/tools-gmail

Google Calendar: https://llama-hub-ui.vercel.app/l/tools-google_calendar

Google Search: https://llama-hub-ui.vercel.app/l/tools-google_search

**Note**: Both Gmail and Google Calendar require a credentials.json file. In the Colab notebook environment, make sure to upload the relevant credential files.


In [ ]:
from llama_hub.tools.gmail.base import GmailToolSpec
from llama_hub.tools.google_calendar.base import GoogleCalendarToolSpec
from llama_hub.tools.google_search.base import GoogleSearchToolSpec

gmail_tools = GmailToolSpec().to_tool_list()
gcal_tools = GoogleCalendarToolSpec().to_tool_list()
gsearch_tools = GoogleSearchToolSpec(key="<insert_api_key>", engine="<insert_engine>").to_tool_list()



Let's take a look at all of the tools we have available from the 3 tool specs we initialized:

In [ ]:
for tool in [*gmail_tools, *gcal_tools, *gsearch_tools]:
    print(tool.metadata.name)
    print(tool.metadata.description)

We have to be conscious of the models context length when using these tools as if we are not careful the response can easily be larger than the token limit. In particular, the load_data function for emails returns large payloads, as well as google search. In this example I will wrap those two tools in the Load and Search Meta tool:

In [ ]:

from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

print('Wrapping ' + gsearch_tools[0].metadata.name)
gsearch_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
    gsearch_tools[0],
).to_tool_list()

print('Wrapping gmail ' + gmail_tools[0].metadata.name)
gmail_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
    gmail_tools[0],
).to_tool_list()

print('Wrapping google calendar ' + gcal_tools[0].metadata.name)
gcal_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
    gcal_tools[0],
).to_tool_list()


Notice we are only wrapping individual tools out of the tool list. Lets combine the all the tools together into a combined list:

In [ ]:
all_tools = [*gsearch_load_and_search_tools, *gmail_load_and_search_tools, *gcal_load_and_search_tools, *gcal_tools[1::], *gmail_tools[1::], *gsearch_tools[1::]]

Now the tools are ready to pass to the agent:

In [ ]:
agent = OpenAIAgent.from_tools(all_tools, verbose=True)

## Interacting with the Agent

We are now ready to interact with the Agent and test the calendar, email and search capabilities! Let's try out the search first:

In [ ]:
response = agent.chat(
    'search google and find the email address for a dentist in toronto near bloor and dufferin'
)
print(str(response))

=== Calling Function ===
Calling function: google_search with args: {
  "query": "dentist in Toronto near Bloor and Dufferin email address"
}
Got output: Content loaded! You can now search the information using read_google_search
=== Calling Function ===
Calling function: read_google_search with args: {
  "query": "email address for dentist in Toronto near Bloor and Dufferin"
}



KeyboardInterrupt



In [ ]:
response = agent.chat(
    'Can you create an email to the dentist inquiring if ' +
    'they have any availablity at 4pm anytime next week'
)
print(str(response))

In [ ]:
response = agent.chat('Can you replace "Dear Dentist" with "Hello,"')
print(str(response))

In [ ]:
response = agent.chat('Is there any events on my calendar next week at 4pm?')
print(str(response))

In [ ]:
agent.chat('send the email')

## Summary

We were able to use the google search, email and calendar tools to find a dentist and request an appointment, making sure any calendar conflicts were avoided. This notebook should prove useful for experimenting with more complicated agents, combining together the tools available in LlamaHub to create more complicated workflows the agent can execute.

